In [1]:
!pip install lxml

Looking in indexes: https://pypi.org/simple, https://artifacts.deez.re/repository/python-research/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 115.2 MB/s eta 0:00:0000:01

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [115]:
!python parse_xml.py --source_path /data/datasets/drama_gutenberg_no_collections/xmls/ --dest_path /data/datasets/drama_gutenberg_no_collections/full_data.json

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


100%|█████████████████████████████████████████| 293/293 [01:17<00:00,  3.77it/s]
Total number of plays 293
Parsed number of plays 228


In [69]:
!python split_data.py --source_data "/data/datasets/all_drama_gutenberg/scene_data.json" --dest_folder "scene_data/" --min_num_quotes 16 --max_char_in_segment 20


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[TOTAL] # remaining plays: 179
[TRAIN]
# acts: 1507	 # plays: 133	 # unique character: 5392	 avg # speaker in act: 3.6	 total # of quotes: 263270
[VALIDATION]
# acts: 240	 # plays: 17	 # unique character: 1557	 avg # speaker in act: 6.5	 total # of quotes: 50670
[TEST]
# acts: 203	 # plays: 19	 # unique character: 1319	 avg # speaker in act: 6.5	 total # of quotes: 41830
[TRAIN]
# queries: 5392	 avg # target/queries: 4.989614243323442
[VALIDATION]
# queries: 1557	 avg # target/queries: 8.788053949903661
[TEST]
# queries: 1319	 avg # target/queries: 8.669446550416982


In [316]:
!python split_data.py --source_data "/data/datasets/all_drama_gutenberg/full_data.json" --dest_folder "full_data/" --min_num_quotes 16


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[TOTAL] # remaining plays: 303
[TRAIN]
# acts: 226	 # plays: 226	 # unique character: 4109	 avg # speaker in act: 18.2	 total # of quotes: 449407
[VALIDATION]
# acts: 30	 # plays: 30	 # unique character: 917	 avg # speaker in act: 30.6	 total # of quotes: 63934
[TEST]
# acts: 31	 # plays: 31	 # unique character: 1214	 avg # speaker in act: 39.2	 total # of quotes: 74738
[TRAIN]
# queries: 4109	 avg # target/queries: 90.68264784619129
[VALIDATION]
# queries: 917	 avg # target/queries: 55.05016357688113
[TEST]
# queries: 1214	 avg # target/queries: 108.52718286655684


In [26]:
import sys 
sys.path.append("src/")
from tqdm import tqdm
import torch
import numpy as np
from collections import defaultdict
from sklearn.metrics import pairwise_distances
from sklearn.metrics import roc_auc_score
import json
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModel
# from src.utilities.metric import ranking
from src.models.huggingface_transformer import Transformer

In [27]:
def ranking(queries, 
            targets,
            query_authors, 
            target_authors, 
            q_indices,
            t_indices,
            q_play_idx,
            t_play_idx,
            metric='cosine', 
):
    num_queries = len(query_authors)
    ranks = np.zeros((num_queries), dtype=np.float32)
    reciprocal_ranks = np.zeros((num_queries), dtype=np.float32)
    
    distances = pairwise_distances(queries, Y=targets, metric=metric, n_jobs=-1)

    auc_scores = []
    play_scores = defaultdict(list)
    for i in range(num_queries):
        # Get the right targets for current query
        target_index = q_indices[i]
        val_targets = np.where(t_indices == target_index)[0]
        dist = distances[i,val_targets]
        labels = [0] * len(val_targets)
        idx = np.where(target_authors[val_targets] == query_authors[i])[0].item()
        labels[idx] = 1
        
        play_idx = q_play_idx[i] 
        play_scores[play_idx].append(roc_auc_score(labels, 1-dist))
        # sorted_indices = np.argsort(dist)
        # sorted_target_authors = target_authors[sorted_indices]
        # ranks[i] = np.where(sorted_target_authors ==
        #                     query_authors[i])[0].item()
        # reciprocal_ranks[i] = 1.0 / float(ranks[i] + 1)
    scores_per_play = {k : np.mean(v) for k,v in play_scores.items()}
    return_dict = {
        "AUC_mean" : np.mean(list(scores_per_play.values())),
        "AUC_std" : np.std(list(scores_per_play.values())),
        # 'R@2': np.sum(np.less_equal(ranks, 2)) / np.float32(num_queries),
        # 'R@8': np.sum(np.less_equal(ranks, 8)) / np.float32(num_queries),
        # 'R@16': np.sum(np.less_equal(ranks, 16)) / np.float32(num_queries),
        # 'R@32': np.sum(np.less_equal(ranks, 32)) / np.float32(num_queries),
        # 'R@64': np.sum(np.less_equal(ranks, 64)) / np.float32(num_queries),
        # 'MRR': np.mean(reciprocal_ranks)
    }

    return return_dict, scores_per_play

In [4]:
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-mpnet-base-v2")

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [28]:
@torch.no_grad()
def get_auc(queries, targets) : 
    model = SentenceTransformer("all-mpnet-base-v2", device="cuda:0")
    model.max_seq_length = 64
    query_embeddings = []
    q_authors = []
    q_act_index = []
    q_play_idx = []
    for tq in tqdm(queries) : 
        query_embeddings.append(model.encode(list(tq["data"].values())[0], convert_to_tensor=True, device="cuda:0").mean(0).cpu().unsqueeze(0))
        q_authors.append(list(tq["data"].keys())[0])
        q_act_index.append(tq["act_index"])
        q_play_idx.append(tq["play_index"])

    query_embeddings = torch.cat(query_embeddings).numpy()
    
    target_embeddings = []
    t_authors = []
    t_act_index = []
    t_play_idx = []
    for tt in tqdm(targets) : 
        target_embeddings.append(model.encode(list(tt["data"].values())[0], convert_to_tensor=True, device="cuda:0").mean(0).cpu().unsqueeze(0))
        t_authors.append(list(tt["data"].keys())[0])
        t_act_index.append(tt["act_index"])
        t_play_idx.append(tt["play_index"])
    target_embeddings = torch.cat(target_embeddings).numpy()
    
    return ranking(query_embeddings, target_embeddings, np.array(q_authors), np.array(t_authors), np.array(q_act_index), np.array(t_act_index), np.array(q_play_idx),  np.array(t_play_idx))

device = torch.device("cuda:0")

def tokenize(tokenizer, quotes, batch_first = False, max_length=64) : 
    tokens = tokenizer(quotes, max_length = max_length, return_tensors = "pt", truncation=True, padding="max_length")

    if not batch_first:  
        tokens["input_ids"] = tokens["input_ids"].reshape(1, -1, max_length)
        tokens["attention_mask"] = tokens["attention_mask"].reshape(1, -1, max_length)
    else : 
        tokens["input_ids"] = tokens["input_ids"].reshape(-1, 1, max_length)
        tokens["attention_mask"] = tokens["attention_mask"].reshape(-1, 1, max_length)

    return tokens

@torch.no_grad()
def get_luar_auc(queries, targets, tokenizer, model ) : 
    
    query_embeddings = []
    q_authors = []
    q_act_index = []
    q_play_idx = []

    for tq in tqdm(test_queries) : 
        tokens = tokenize(tokenizer, list(tq["data"].values())[0], batch_first=False, max_length=64)
        query_embeddings.append(model(**tokens.to(device)).cpu())
        q_authors.append(list(tq["data"].keys())[0])
        q_act_index.append(tq["act_index"])
        q_play_idx.append(tq["play_index"])

    query_embeddings = torch.cat(query_embeddings).numpy()
    
    target_embeddings = []
    t_authors = []
    t_act_index = []
    t_play_idx = []
    for tt in tqdm(test_targets) : 
        tokens = tokenize(tokenizer, list(tt["data"].values())[0], batch_first=False, max_length=64)
        target_embeddings.append(model(**tokens.to(device)).cpu())
        t_authors.append(list(tt["data"].keys())[0])
        t_act_index.append(tt["act_index"])
        t_play_idx.append(tt["play_index"])

    target_embeddings = torch.cat(target_embeddings).numpy()
    
    return ranking(query_embeddings, target_embeddings, np.array(q_authors), np.array(t_authors), np.array(q_act_index), np.array(t_act_index), np.array(q_play_idx),  np.array(t_play_idx))



## Scenes

In [29]:

import json
test_queries = json.load(open("scene_data/test_queries.json"))
test_targets = json.load(open("scene_data/test_targets.json"))
play_index = json.load(open("scene_data/test_play_index.json"))
# val_queries = json.load(open("scene_data/val_queries.json"))
# val_targets = json.load(open("scene_data/val_targets.json"))

### SBERT

In [323]:
auc,_ = get_auc(test_queries, test_targets)
print(f"SBERT: {auc}")

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1319/1319 [00:45<00:00, 29.02it/s]


SBERT: {'AUC_mean': 0.7200616193930536, 'AUC_std': 0.049477914452999344}


### UAR

In [324]:
tokenizer = AutoTokenizer.from_pretrained("rrivera1849/LUAR-MUD", trust_remote_code=True)
model = AutoModel.from_pretrained("rrivera1849/LUAR-MUD", trust_remote_code=True).to(device)
model.eval()
auc,_ = get_luar_auc(test_queries, test_targets, tokenizer, model)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1319/1319 [00:42<00:00, 30.76it/s]


In [325]:
print(f"UAR: {auc}")

UAR: {'AUC_mean': 0.686006963435264, 'AUC_std': 0.07072840989813857}


### LAR Scene


In [31]:
tokenizer = AutoTokenizer.from_pretrained("gasmichel/UAR_scene", trust_remote_code=True)
model = AutoModel.from_pretrained("gasmichel/UAR_scene", trust_remote_code=True)
model.eval() 
model=model.to(device)

A new version of the following files was downloaded from https://huggingface.co/gasmichel/UAR_scene:
- config.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.py:   0%|          | 0.00/3.77k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/gasmichel/UAR_scene:
- model.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


In [32]:

auc, scores_per_play = get_luar_auc(test_queries, test_targets, tokenizer, model)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1319/1319 [00:17<00:00, 76.86it/s]


In [33]:
print(f"LAR-scene: {auc}")

LAR-scene: {'AUC_mean': np.float64(0.8232535514803928), 'AUC_std': np.float64(0.060217817506079796)}


In [352]:
d = json.load(open("/data/datasets/all_drama_gutenberg/scene_data.json"))

In [357]:
d = json.load(open("/data/datasets/all_drama_gutenberg/scene_data.json"))
sp  = sorted(scores_per_play.items(), key=lambda x:x[1], reverse=True)
rp = {v:k for k,v in play_index.items()}
for idx, score in sp : 

    try : 
        play = rp[idx]
        author = d[play]["metadata"]["author"]
        print(f"{play} & {author} & {score * 100 : 0.1f}\\\\")
    except : 
        pass

Justice & John Galsworthy &  95.6\\
The Alchemist & Ben Jonson &  91.8\\
Hamlet & William Shakespeare &  86.5\\
Plays : Third Series & John Galsworthy &  86.3\\
The works of John Dryden, $c now first collected in eighteen volumes. $p Volume 07 & John Dryden &  85.2\\
The Life of Timon of Athens & William Shakespeare &  84.2\\
King Henry the Fifth	Arranged for Representation at the Princess's Theatre & William Shakespeare &  83.7\\
Night Must Fall : a Play in Three Acts & Emlyn Williams &  83.7\\
The Spanish Tragedy & Thomas Kyd &  83.2\\
The works of John Dryden, $c now first collected in eighteen volumes. $p Volume 06 & John Dryden &  83.0\\
The Scarlet Stigma: A Drama in Four Acts & Nathaniel Hawthorne &  82.4\\
King John & William Shakespeare &  81.8\\
The Works of Charles and Mary Lamb — Volume 4	Poems and Plays & Mary Lamb &  80.8\\
The Scornful Lady & John Fletcher &  80.8\\
Married Life	A Comedy, in Three Acts & John Baldwin Buckstone &  80.0\\
The Flutter of the Goldleaf; and O

## Play

In [34]:

import json
test_queries = json.load(open("full_data/test_queries.json"))
test_targets = json.load(open("full_data/test_targets.json"))
play_index = json.load(open("full_data/test_play_index.json"))


### SBERT

In [7]:
auc,_ = get_auc(test_queries, test_targets)
print(f"SBERT: {auc}")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/usr/local/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1214/1214 [00:39<00:00, 31.02it/s]


SBERT: {'AUC_mean': 0.7880883961207349, 'AUC_std': 0.10085498993692368}


### UAR

In [8]:
tokenizer = AutoTokenizer.from_pretrained("rrivera1849/LUAR-MUD", trust_remote_code=True)
model = AutoModel.from_pretrained("rrivera1849/LUAR-MUD", trust_remote_code=True).to(device)
model.eval()
auc,_ = get_luar_auc(test_queries, test_targets, tokenizer, model)
print(f"UAR: {auc}")

tokenizer_config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/330M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/328M [00:00<?, ?B/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1214/1214 [00:26<00:00, 46.41it/s]


UAR: {'AUC_mean': 0.7445067504593709, 'AUC_std': 0.10817062527677136}


### LAR - Play

In [36]:
tokenizer = AutoTokenizer.from_pretrained("gasmichel/UAR_play", trust_remote_code=True)
model = AutoModel.from_pretrained("gasmichel/UAR_play", trust_remote_code=True)
model.eval() 
model=model.to(device)

model.safetensors:   6%|6         | 21.0M/330M [00:00<?, ?B/s]

In [37]:
auc, scores_per_play = get_luar_auc(test_queries, test_targets, tokenizer, model)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1214/1214 [00:28<00:00, 41.96it/s]


In [38]:
print(f"LAR-play: {auc}")

LAR-play: {'AUC_mean': np.float64(0.8414877987109732), 'AUC_std': np.float64(0.08849851536808165)}


In [13]:
d = json.load(open("/data/datasets/all_drama_gutenberg/full_data.json"))
sp  = sorted(scores_per_play.items(), key=lambda x:x[1], reverse=True)
rp = {v:k for k,v in play_index.items()}
for idx, score in sp : 

    try : 
        play = rp[idx]
        author = d[play]["metadata"]["author"]
        print(f"{play} & {author} & {score * 100 : 0.1f}\\\\")
    except : 
        pass

Bride Roses & William Dean Howells &  100.0\\
Fanny's First Play & Bernard Shaw &  100.0\\
Five Little Plays & Alfred Sutro &  98.6\\
John Bull's Other Island & Bernard Shaw &  95.4\\
Mrs. Dot	A Farce & W. Somerset (William Somerset) Maugham &  94.5\\
The Earl of Essex: A Tragedy, in Five Acts & Henry Jones &  94.4\\
The Way of the World & William Congreve &  92.7\\
Plays : Fifth Series & John Galsworthy &  92.5\\
Representative Plays by American Dramatists: 1856-1911: The New York Idea & Langdon Elwyn Mitchell &  91.9\\
Plays: Lady Frederick, The Explorer, A Man of Honour & W. Somerset (William Somerset) Maugham &  90.3\\
Great Catherine & Bernard Shaw &  89.3\\
Hypolympia	Or, The Gods in the Island, an Ironic Fantasy & Edmund Gosse &  87.1\\
The works of John Dryden, $c now first collected in eighteen volumes. $p Volume 06 & John Dryden &  87.0\\
The Grecian Daughter & Arthur Murphy &  85.5\\
The Works of Lord Byron. Vol. 5	Poetry & ? ? &  85.2\\
Dramatic Technique & George Pierce Ba